In [496]:
cd C:\Users\andre\Projects\rtsp\for_labelling

C:\Users\andre\Projects\rtsp\for_labelling


In [497]:
import tensorflow as tf

In [498]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [499]:
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import shutil
import numpy as np
import pandas as pd

In [500]:
path_images = glob.glob('*')

In [501]:
cd output

C:\Users\andre\Projects\rtsp\for_labelling\output


In [502]:
labelled = pd.read_csv('assigned_classes_automatically_generated.csv')

In [503]:
labelled['w'] = np.nan
labelled['h'] = np.nan
labelled['aspect'] = np.nan
for i, row in labelled.iterrows():
    w, h = list(map(int, row['img'].replace('h', '').replace('w', '').split('_')[-3:-1]))
    asp = h/w
    labelled.iloc[i, -3] = w  # w
    labelled.iloc[i, -2] = h  # h
    labelled.iloc[i, -1] = asp  # aspect

In [504]:
more ../../labels.txt

discard
not_aceptable_size_aspect_cam_noise
aceptable_size_aspects_NOT_valid_being
aceptable_size_aspects_YES_valid_being

In [505]:
notacceptable = labelled[labelled['not_aceptable_size_aspect_cam_noise'] == 1]['aspect']
np.percentile(notacceptable, [10, 50, 90])

array([0.08472876, 0.35974974, 0.74222718])

In [506]:
acceptable = labelled[labelled['not_aceptable_size_aspect_cam_noise'] != 1]['aspect']

In [507]:
np.percentile(acceptable, [5, 50, 90])

array([0.43340884, 1.        , 2.68551913])

In [508]:
X = labelled[['w', 'h', 'aspect']].to_numpy()

In [509]:
y = labelled['not_aceptable_size_aspect_cam_noise'].to_numpy()

In [510]:
len(X)

219

In [511]:
X_t, y_t = X[:100], y[:100]

## Noise Classifier - sklearn

In [512]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

In [513]:
etree = ExtraTreesClassifier()
etree = RandomForestClassifier(verbose=True)

In [514]:
etree.fit(X_t, y_t)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


RandomForestClassifier(verbose=True)

In [515]:
etree.score(X[100:], y[100:])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


0.9663865546218487

In [516]:
etree.feature_importances_

array([0.4570906 , 0.19035907, 0.35255033])

Not Noise - even very vertical

In [517]:
%%time
Xs = [[100., 800., 800./100.]]
print(Xs)
etree.predict_proba(Xs)

[[100.0, 800.0, 8.0]]
Wall time: 17 ms


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


array([[0.81, 0.19]])

Noise - very horizontal

In [518]:
Xs = [[900., 150., 150./900.]]
print(Xs)
etree.predict_proba(Xs)

[[900.0, 150.0, 0.16666666666666666]]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


array([[0.27, 0.73]])

In [519]:
etree.classes_

array([0, 1], dtype=int64)

In [520]:
import joblib

joblib.dump(etree, "pcam_noise_rf.joblib")

['pcam_noise_rf.joblib']

## Noise Classifier - tensorflow

In [538]:
import tensorflow  as tf
from tensorflow.keras.callbacks import EarlyStopping

In [647]:
X.shape, y.shape

((219, 3), (219,))

In [648]:
ixs = numpy.random.randint(len(X), size=len(X))

In [649]:
X = X[ixs]
y = y[ixs]

In [1044]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(3, 1)),
    tf.keras.layers.Dense(300, activation='sigmoid'),         
    tf.keras.layers.Dropout(0.3),    
    tf.keras.layers.Dense(50, activation='sigmoid'),             
    tf.keras.layers.Dense(1, activation='sigmoid'),                   
])

In [1045]:
#ErlyStopping?

In [1046]:
early_stopping = EarlyStopping('val_loss', 0.005, patience=30, verbose=2, restore_best_weights=True)

In [1047]:
model.compile(optimizer='adam',
               loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [1048]:
model.reset_states()

In [1055]:
model.fit(X[:100], y[:100], validation_data=(X[100:], y[100:]), batch_size=80, epochs=1000, verbose=1,
          callbacks=[early_stopping])#, validation_split=0.5)

Epoch 1/1000
2/2 [==============================] - 0s 112ms/step - loss: 0.2065 - accuracy: 0.9200 - val_loss: 0.2297 - val_accuracy: 0.8739
Epoch 2/1000
2/2 [==============================] - 0s 50ms/step - loss: 0.2121 - accuracy: 0.9300 - val_loss: 0.2370 - val_accuracy: 0.8908
Epoch 3/1000
2/2 [==============================] - 0s 54ms/step - loss: 0.2169 - accuracy: 0.9100 - val_loss: 0.2328 - val_accuracy: 0.8739
Epoch 4/1000
2/2 [==============================] - 0s 53ms/step - loss: 0.2165 - accuracy: 0.9400 - val_loss: 0.2201 - val_accuracy: 0.9160
Epoch 5/1000
2/2 [==============================] - 0s 50ms/step - loss: 0.1904 - accuracy: 0.9300 - val_loss: 0.2211 - val_accuracy: 0.9496
Epoch 6/1000
2/2 [==============================] - 0s 56ms/step - loss: 0.2207 - accuracy: 0.9600 - val_loss: 0.2183 - val_accuracy: 0.9160
Epoch 7/1000
2/2 [==============================] - 0s 53ms/step - loss: 0.2343 - accuracy: 0.9200 - val_loss: 0.2197 - val_accuracy: 0.8908
Epoch 8/1000

In [1058]:
model.evaluate(X[100:], y[100:])

4/4 [==============================] - 0s 5ms/step - loss: 0.2127 - accuracy: 0.9160


[0.2127009630203247, 0.9159663915634155]

In [1059]:
model.evaluate(X, y)

7/7 [==============================] - 0s 4ms/step - loss: 0.1947 - accuracy: 0.9315


[0.1946951001882553, 0.931506872177124]

In [1060]:
len(model.weights)

6

In [1061]:
model.weights[0].shape

TensorShape([3, 300])

In [1062]:
model.weights[1].shape

TensorShape([300])

In [1063]:
model.weights[2].shape

TensorShape([300, 50])

In [1064]:
model.weights[3].shape

TensorShape([50])

In [1065]:
model.weights[4].shape

TensorShape([50, 1])

In [1066]:
model.weights[5]

<tf.Variable 'dense_407/bias:0' shape=(1,) dtype=float32, numpy=array([-0.01720193], dtype=float32)>

In [1067]:
pwd

'C:\\Users\\andre\\Projects\\rtsp\\for_labelling\\output'

In [1070]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\andre\AppData\Local\Temp\tmptpa2hznp\assets


In [1072]:
len(tflite_model)

67556

In [1073]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)